In [10]:
import pathlib
import os
from pathlib import Path
import sys
 
if sys.argv:
    sys.path.insert(0, str(pathlib.Path(os.path.dirname(os.path.abspath(""))).resolve()))
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
sys.path

['/home/ogrobertino/UpliftModelingResearch',
 PosixPath('/home/ogrobertino/UpliftModelingResearch'),
 '/Users/ogrobertino/UpliftModelingResearch',
 '/home/ogrobertino/.pyenv/versions/3.10.6/lib/python310.zip',
 '/home/ogrobertino/.pyenv/versions/3.10.6/lib/python3.10',
 '/home/ogrobertino/.pyenv/versions/3.10.6/lib/python3.10/lib-dynload',
 '',
 '/home/ogrobertino/test_env/lib/python3.10/site-packages']

In [12]:
import torch

In [26]:
from src.models.NNUpliftModeling.NNUpliftModeling import NNUpliftModeling 

In [27]:
input_dim = 20
descn_config = {
        'input_dim': input_dim,
        'hidden_dim': 64,
        'latent_dim': 32,
        'dropout_rate': 0.2,
        'batch_size': 32,
        'epochs': 50,
        'early_stopping_patience': 5,
        'alpha': 0.5,  # Вес для KL-регуляризации
        'beta': 0.1,   # Вес для балансировочной потери
        'gamma': 0.01, # Вес для дополнительной регуляризации
        'optimizer': {
            'name': 'Adam',
            'lr': 0.001,
            'weight_decay': 0.0001
        },
        'scheduler': {
            'name': 'ReduceLROnPlateau',
            'patience': 3,
            'factor': 0.5
        }
    }

In [28]:
NNUpliftModeling(descn_config)

AttributeError: 'NoneType' object has no attribute 'parameters'

In [ ]:
    # Создание синтетических данных для демонстрации
    import numpy as np
    
    # Параметры данных
    num_samples = 1000
    input_dim = 20
    
    # Генерация признаков
    X = np.random.normal(0, 1, (num_samples, input_dim))
    
    # Генерация индикатора воздействия
    T = np.random.binomial(1, 0.5, num_samples)
    
    # Генерация исходов
    # Базовый исход
    Y0 = 0.1 * np.sum(X[:, :5], axis=1) + np.random.normal(0, 0.5, num_samples)
    # Эффект воздействия
    tau = 0.2 * np.sum(X[:, 5:10], axis=1) + 1.0 + np.random.normal(0, 0.5, num_samples)
    # Наблюдаемый исход
    Y = Y0 + T * tau
    
    # Преобразование в тензоры PyTorch
    X_tensor = torch.tensor(X, dtype=torch.float32)
    T_tensor = torch.tensor(T, dtype=torch.float32)
    Y_tensor = torch.tensor(Y, dtype=torch.float32)
    
    # Создание TensorDataset
    dataset = TorchDataset(X_tensor, T_tensor, Y_tensor)
    
    # Разделение на обучающую и тестовую выборки
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
    
    # Пример конфигурации для DESCN
    descn_config = {
        'input_dim': input_dim,
        'hidden_dim': 64,
        'latent_dim': 32,
        'dropout_rate': 0.2,
        'batch_size': 32,
        'epochs': 50,
        'early_stopping_patience': 5,
        'alpha': 0.5,  # Вес для KL-регуляризации
        'beta': 0.1,   # Вес для балансировочной потери
        'gamma': 0.01, # Вес для дополнительной регуляризации
        'optimizer': {
            'name': 'Adam',
            'lr': 0.001,
            'weight_decay': 0.0001
        },
        'scheduler': {
            'name': 'ReduceLROnPlateau',
            'patience': 3,
            'factor': 0.5
        }
    }
    
    # Создание и обучение модели DESCN
    descn_model = DESCNUplift(config_json=descn_config)
    descn_history = descn_model.fit(train_dataset)
    
    # Оценка на тестовых данных
    descn_predictions = descn_model.predict(test_dataset)
    
    # Пример конфигурации для EFIN
    efin_config = {
        'input_dim': input_dim,
        'hidden_dim': 64,
        'interaction_layers': 2,
        'num_heads': 4,
        'dropout_rate': 0.2,
        'batch_size': 32,
        'epochs': 50,
        'early_stopping_patience': 5,
        'lambda_reg': 0.01,  # Вес для L2-регуляризации
        'optimizer': {
            'name': 'Adam',
            'lr': 0.001,
            'weight_decay': 0.0001
        },
        'scheduler': {
            'name': 'ReduceLROnPlateau',
            'patience': 3,
            'factor': 0.5
        }
    }
    
    # Создание и обучение модели EFIN
    efin_model = EFINUplift(config_json=efin_config)
    efin_history = efin_model.fit(train_dataset)
    
    # Оценка на тестовых данных
    efin_predictions = efin_model.predict(test_dataset)
    
    # Сравнение производительности моделей
    descn_timing = descn_model.measure_inference_time(test_dataset, batch_size=64)
    efin_timing = efin_model.measure_inference_time(test_dataset, batch_size=64)
    
    print("\nInference Performance Comparison:")
    print(f"DESCN: {descn_timing['examples_per_second']:.2f} examples/second")
    print(f"EFIN: {efin_timing['examples_per_second']:.2f} examples/second")
    
    # Сохранение моделей
    descn_model.save("models/descn_uplift_model.pt")
    efin_model.save("models/efin_uplift_model.pt")
    
    print("\nModels saved successfully.")